***This is Unsupervised ML recommendation model. It is a unsupervised because , in this model we don't use any kind of labeling , we just try to find out the pattern with the help of consine similiraty.***

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

Dataset URL: https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset
License(s): unknown
  0% 0.00/195M [00:00<?, ?B/s]
100% 195M/195M [00:00<00:00, 2.20GB/s]


In [3]:
!unzip /content/movielens-20m-dataset.zip

Archive:  /content/movielens-20m-dataset.zip
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              
  inflating: tag.csv                 


In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [5]:
rating_df = pd.read_csv("/content/rating.csv")
rating_df = rating_df.sample(500000,random_state=42)

In [6]:
rating_df.shape

(500000, 4)

In [7]:
rating_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [8]:
movie_df = pd.read_csv("/content/movie.csv")
movie_df = movie_df[movie_df['movieId'].isin(rating_df['movieId'])]

In [9]:
movie_df.shape

(13152, 3)

In [10]:
movie_user_matrix = rating_df.pivot_table(index='userId',columns='movieId',values='rating')

In [11]:
len(rating_df['userId'].unique())

106761

In [12]:
movie_user_matrix.shape

(106761, 13152)

In [13]:
movie_user_matrix = movie_user_matrix.fillna(0)

In [14]:
movie_user_matrix[1].unique()

array([0. , 3. , 2.5, 3.5, 5. , 4.5, 4. , 2. , 1. , 1.5, 0.5])

In [15]:
# in sklean it computes the similarity within rows,
# so, we need to transpose our matrix


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# Compare movies based on users’ ratings

In [18]:
movie_similarity = cosine_similarity(movie_user_matrix.T)

In [19]:
df_movie_similarity = pd.DataFrame(movie_similarity,index = movie_user_matrix.columns,columns = movie_user_matrix.columns) # compare the movies to every movie and find the similirity score

In [20]:
df_movie_similarity.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129421,129428,129915,129927,130326,130340,130502,130512,130970,131092
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.007916,0.001603,0.000000,0.006740,0.016661,0.008531,0.000000,0.002660,0.007066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.007916,1.000000,0.000000,0.004681,0.002100,0.004699,0.011359,0.000000,0.004696,0.005389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001603,0.000000,1.000000,0.000000,0.007778,0.007519,0.002434,0.006773,0.002088,0.009227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.004681,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.006740,0.002100,0.007778,0.000000,1.000000,0.000000,0.013294,0.000000,0.011116,0.005233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_movie_similarity.shape

(13152, 13152)

In [22]:
def recommend_similar_movies(movie_id, n=5):
    similar_scores = df_movie_similarity[movie_id].sort_values(ascending=False)

    top_movies = similar_scores.iloc[1:n+1]

    recommended = movie_df[movie_df['movieId'].isin(top_movies.index)]
    return recommended[['movieId', 'title', 'genres']]


In [23]:
recommend_similar_movies(1)


,movieId,title,genres
4697,4793,Montenegro (1981),Comedy|Drama
5156,5253,Cousins in Love (a.k.a. Tender Cousins) (Tendr...,Drama|Romance
6715,6825,"Reluctant Astronaut, The (1967)",Comedy
9623,31026,"Phantom of the Opera, The (1989)",Drama|Horror|Musical
10723,42543,She (1935),Adventure|Fantasy|Romance


In [24]:
def get_movie_id(title):
    return movie_df[movie_df['title'].str.contains(title, case=False, regex=False)]['movieId'].values[0]


In [25]:
movie_id = get_movie_id("Toy Story")
recommend_similar_movies(movie_id)


,movieId,title,genres
4697,4793,Montenegro (1981),Comedy|Drama
5156,5253,Cousins in Love (a.k.a. Tender Cousins) (Tendr...,Drama|Romance
6715,6825,"Reluctant Astronaut, The (1967)",Comedy
9623,31026,"Phantom of the Opera, The (1989)",Drama|Horror|Musical
10723,42543,She (1935),Adventure|Fantasy|Romance
